In [1]:
from utils.BackTest import Engine, Strategy
import numpy as np
import pandas as pd

In [2]:
df_1 = pd.read_csv('../data/processed/merged_df.csv')
df_2 = pd.read_csv('../data/processed/features_trading_strategy.csv')

In [3]:
df_1

,datetime,EMA_8,EMA_13,EMA_21,EMA_signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Hist,...,OBV_Signal,ATR,reddit_smoothed_sentiment_weibull,return,hourly_volatility,TOTAL_TRADES,VOLUME,smoothed_sentiment,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
0,2024-03-02 02:00:00+00:00,62220.598138,62172.294796,62076.132135,1,62184.157168,62011.323303,172.833864,187.562435,-14.728570,...,1,384.756667,0.157746,0.001309,0.006053,56088,1153.46042,-222.074588,-188.470966,0.000000
1,2024-03-02 03:00:00+00:00,62239.898552,62191.602682,62097.161032,1,62203.125296,62033.258614,169.866682,184.023284,-14.156603,...,0,347.451667,0.092267,0.003160,0.004931,67185,1465.47878,-204.266170,-204.409405,0.552411
2,2024-03-02 04:00:00+00:00,62194.576652,62169.366585,62091.596392,1,62177.406020,62033.457976,143.948043,176.008236,-32.060193,...,-1,333.268333,0.050600,-0.004357,0.005180,56098,1147.71004,-234.015147,-205.344729,0.000000
3,2024-03-02 05:00:00+00:00,62152.792951,62146.107073,62083.864902,1,62151.120478,62031.464793,119.655685,164.737726,-45.082040,...,0,326.113333,0.026060,-0.000474,0.004688,65329,1415.65823,-225.408115,-200.706776,0.000000
4,2024-03-02 06:00:00+00:00,62106.838962,62117.520348,62071.331729,0,62119.563482,62025.134067,94.429414,150.676063,-56.246649,...,0,325.211667,0.012619,-0.000977,0.004189,57826,1059.53378,-208.216550,-185.091972,1.010990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,2025-01-26 07:00:00+00:00,104968.753394,104930.387351,104892.740104,1,104936.968508,104874.604967,62.363541,41.106721,21.256820,...,1,298.715000,0.055311,0.000072,0.001646,41775,208.35672,30.142278,-13.564028,0.000000
8377,2025-01-26 08:00:00+00:00,104982.452640,104944.674872,104905.254640,1,104951.342584,104886.145340,65.197244,45.924826,19.272419,...,0,325.368333,0.030398,0.000305,0.001628,63281,283.50224,35.082025,-6.781369,0.103237
8378,2025-01-26 09:00:00+00:00,104904.136498,104899.725605,104880.234218,1,104901.909879,104867.173833,34.736046,43.687070,-8.951024,...,-1,326.935000,0.015620,-0.003812,0.001915,91103,621.23723,37.058349,0.681248,0.000000
8379,2025-01-26 10:00:00+00:00,104862.170609,104873.377661,104865.239289,-1,104873.199128,104855.923179,17.275949,38.404846,-21.128896,...,1,277.221667,0.007520,0.000815,0.001946,50517,252.60356,33.197390,8.190720,0.000000


In [4]:
df_1 = df_1.drop_duplicates(subset=['datetime'], keep='first')

In [5]:
df_2

,datetime,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,VOLUME,VOLUME_BUY,...,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR
0,2023-09-12 08:00:00+00:00,25822.71,25871.06,25735.29,25766.93,28391,12051,16340,1893.94969,846.27631,...,NaN,0,NaN,NaN,0,135.77,NaN,NaN,135.77,NaN
1,2023-09-12 09:00:00+00:00,25766.93,25875.85,25736.12,25869.97,29851,14231,15620,1455.35759,719.08619,...,NaN,0,1455.35759,1455.357590,0,139.73,108.92,30.81,139.73,NaN
2,2023-09-12 10:00:00+00:00,25869.97,26331.38,25841.38,26159.98,104997,53695,51302,6661.53232,3444.38486,...,NaN,0,6661.53232,1951.183755,1,490.00,461.41,28.59,490.00,NaN
3,2023-09-12 11:00:00+00:00,26159.98,26175.97,26031.63,26098.99,36029,15580,20449,2116.39977,923.61832,...,NaN,0,-2116.39977,1563.794848,-1,144.34,15.99,128.35,144.34,NaN
4,2023-09-12 12:00:00+00:00,26098.99,26230.00,26095.40,26121.84,38364,17018,21346,1686.56984,753.80994,...,NaN,0,1686.56984,1575.487704,1,134.60,131.01,3.59,134.60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,2025-03-27 15:00:00+00:00,87297.33,87575.00,86752.58,86974.07,180420,91975,88445,1554.66361,699.72490,...,38.966585,0,-1554.66361,-216.223678,-1,822.42,277.67,544.75,822.42,736.608333
13496,2025-03-27 16:00:00+00:00,86974.07,87271.43,86805.20,86835.24,129979,66364,63615,671.69986,244.03675,...,50.664024,0,-671.69986,-259.602362,0,466.23,297.36,168.87,466.23,732.878333
13497,2025-03-27 17:00:00+00:00,86835.24,87150.95,86796.00,86913.05,120897,50367,70530,596.49192,257.41179,...,42.496856,0,596.49192,-178.069573,1,354.95,315.71,39.24,354.95,703.821667
13498,2025-03-27 18:00:00+00:00,86913.05,87420.00,86876.00,87206.52,103020,44121,58899,473.09632,222.07273,...,53.294442,0,473.09632,-116.053774,0,544.00,506.95,37.05,544.00,730.280000


In [6]:
df_3 = df_1[['datetime','reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']]

In [7]:
df_3[1:].describe()

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
count,7.929000e+03,7929.000000,7929.000000
mean,2.994162e-02,-105.388053,0.304112
std,8.777959e-01,395.040101,0.822927
min,-4.277553e+00,-5128.788745,-3.930538
25%,-3.054081e-01,-257.068523,0.000000
50%,-9.775172e-18,-70.549400,0.000000
75%,2.992846e-01,99.450326,0.000000
max,5.323996e+00,1046.597034,7.955905


In [8]:
#df_4 = pd.merge(df_3, df_2, on='datetime', how='inner')

In [9]:
df_4 = df_2

In [10]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13500 entries, 0 to 13499
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            13500 non-null  object 
 1   OPEN                13500 non-null  float64
 2   HIGH                13500 non-null  float64
 3   LOW                 13500 non-null  float64
 4   CLOSE               13500 non-null  float64
 5   TOTAL_TRADES        13500 non-null  int64  
 6   TOTAL_TRADES_BUY    13500 non-null  int64  
 7   TOTAL_TRADES_SELL   13500 non-null  int64  
 8   VOLUME              13500 non-null  float64
 9   VOLUME_BUY          13500 non-null  float64
 10  VOLUME_SELL         13500 non-null  float64
 11  return              13499 non-null  float64
 12  log_return          13499 non-null  float64
 13  hourly_volatility   13488 non-null  float64
 14  EMA_8               13500 non-null  float64
 15  EMA_13              13500 non-null  float64
 16  EMA_

In [11]:
signs = np.sign(df_4[['reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']])

sentiment_cols = ['reddit_smoothed_sentiment_weibull',
                  'news_smoothed_sentiment_weibull',
                  'tele_smoothed_sentiment_weibull']

# Get the absolute values and shift them to ensure minimum is 1
shifted_abs_values = df_4[sentiment_cols].abs() + (1 - df_4[sentiment_cols].abs().min())

log_transformed = np.log(shifted_abs_values)
log_signed = log_transformed * signs

df_4[['reddit_sentiment_log','news_sentiment_log','tele_sentiment_log']] = log_signed

KeyError: "None of [Index(['reddit_smoothed_sentiment_weibull', 'news_smoothed_sentiment_weibull',\n       'tele_smoothed_sentiment_weibull'],\n      dtype='object')] are in the [columns]"

In [ ]:
df_4[['reddit_sentiment_log','news_sentiment_log','tele_sentiment_log']].describe()

In [ ]:
df_4[['reddit_Signal','news_Signal','tele_Signal']] = np.sign(df_4[['reddit_sentiment_log','news_sentiment_log','tele_sentiment_log']])

In [12]:
df_4['datetime'] = pd.to_datetime(df_4['datetime']).dt.tz_localize(None)
df_4 = df_4.set_index('datetime')

In [13]:
df_4['typical_price'] = (df_4['HIGH'] + df_4['LOW'] + df_4['CLOSE']) / 3

df_4['DATE'] = df_4.index.date  
df_4['VWAP'] = df_4.groupby('DATE').apply(
    lambda g: (g['typical_price'] * g['VOLUME']).cumsum() / g['VOLUME'].cumsum()
).reset_index(level=0, drop=True)

C:\Users\leoqi\AppData\Local\Temp\ipykernel_88956\4284048504.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_4['VWAP'] = df_4.groupby('DATE').apply(


In [14]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return -1
    else:
        return 0

In [15]:
df_4['VWAP_Signal'] = df_4.apply(determine_signal, axis=1)

df_4['VWAP_Signal'] = df_4['VWAP_Signal'].fillna(0)

In [16]:
df_filtered = df_4.rename(columns={'OPEN':'Open',
                                 'CLOSE':'Close',
                                 'HIGH':'High',
                                 'LOW':'Low'})

In [17]:
strategy_dict = {
    'MACD_Signal': 0.00,
    'BB_RSI_Signal': 0.00,
    'EMA_Signal': 0.00,
    'GoldenCross_Signal': 0.00,
    'Stochastic_Signal': 0.00,
    'OBV_Signal': 0.00,
    'VWAP_Signal': 0.00,
    # 'tele_Signal': 0.00,
    # 'reddit_Signal': 0.00,
    # 'news_Signal': 0.00,
}

In [18]:
for name in strategy_dict.keys():
    strategy_dict[name] = 1/len(strategy_dict.keys())

In [19]:
strategy_dict

{'MACD_Signal': 0.14285714285714285,
 'BB_RSI_Signal': 0.14285714285714285,
 'EMA_Signal': 0.14285714285714285,
 'GoldenCross_Signal': 0.14285714285714285,
 'Stochastic_Signal': 0.14285714285714285,
 'OBV_Signal': 0.14285714285714285,
 'VWAP_Signal': 0.14285714285714285}

In [20]:
# class MultiStrategyAggregator(Strategy):
#     def __init__(self,weights):
#         super().__init__()
#         self.weights = weights # as a dictionary of name and weight

#     def on_bar(self):

#         weights_keys = list(self.weights.keys())

#         #map the features to the weights and sum score * weight
#         features = self.data.loc[self.current_idx][weights_keys]

#         pred_score = sum([features[key] * self.weights[key] for key in weights_keys])

#         # if self.current_idx == self.data.index[0]:  # or self.data.index[0] if that's a datetime index
#         #     # Buy in 80% of available cash after subtracting 1% for transaction cost
#         #     size = round((0.80 * (self.cash - (0.001 * self.cash))) / self.close(), 8)
#         #     self.buy("btc", size=size)

#         if pred_score > 0.50:
#             self.buy(
#                 "btc",
#                size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
#             )

In [21]:
class MultiStrategyAggregator(Strategy):
    def __init__(self,weights, bet_size=1, atr_multiplier=1.0, use_leverage=False):
        super().__init__()
        self.weights = weights # as a dictionary of name and weight
        self.bet_size = bet_size
        self.tp_atr_multiplier = atr_multiplier
        self.sl_atr_multiplier = atr_multiplier
        self.use_leverage = use_leverage

    def on_bar(self):

        weights_keys = list(self.weights.keys())

        #map the features to the weights and sum score * weight
        features = self.data.loc[self.current_idx][weights_keys]

        pred_score = sum([features[key] * self.weights[key] for key in weights_keys if features[key]])

        # print(pred_score) comment out for now

        if pred_score > 0.5:
            size = round(0.15 * (self.cash + (self.close() * self.position_size())) /self.close(),8)
            if self.use_leverage:
                size = 2 * size # Using 2x leverage
                self.buy("btc",size = size,)
            else:
                self.buy("btc",size = size,)

In [22]:
df_filtered

,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,VOLUME,VOLUME_BUY,VOLUME_SELL,...,OBV_Signal,H-L,H-PC,L-PC,TR,ATR,typical_price,DATE,VWAP,VWAP_Signal
datetime,,,,,,,,,,,,,,,,,,,,,
2023-09-12 08:00:00,25822.71,25871.06,25735.29,25766.93,28391,12051,16340,1893.94969,846.27631,1047.67338,...,0,135.77,NaN,NaN,135.77,NaN,25791.093333,2023-09-12,25791.093333,-1
2023-09-12 09:00:00,25766.93,25875.85,25736.12,25869.97,29851,14231,15620,1455.35759,719.08619,736.27140,...,0,139.73,108.92,30.81,139.73,NaN,25827.313333,2023-09-12,25806.831827,1
2023-09-12 10:00:00,25869.97,26331.38,25841.38,26159.98,104997,53695,51302,6661.53232,3444.38486,3217.14746,...,1,490.00,461.41,28.59,490.00,NaN,26110.913333,2023-09-12,26009.177371,1
2023-09-12 11:00:00,26159.98,26175.97,26031.63,26098.99,36029,15580,20449,2116.39977,923.61832,1192.78145,...,-1,144.34,15.99,128.35,144.34,NaN,26102.196667,2023-09-12,26025.410745,1
2023-09-12 12:00:00,26098.99,26230.00,26095.40,26121.84,38364,17018,21346,1686.56984,753.80994,932.75990,...,1,134.60,131.01,3.59,134.60,NaN,26149.080000,2023-09-12,26040.509899,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-27 15:00:00,87297.33,87575.00,86752.58,86974.07,180420,91975,88445,1554.66361,699.72490,854.93871,...,-1,822.42,277.67,544.75,822.42,736.608333,87100.550000,2025-03-27,87023.263740,-1
2025-03-27 16:00:00,86974.07,87271.43,86805.20,86835.24,129979,66364,63615,671.69986,244.03675,427.66311,...,0,466.23,297.36,168.87,466.23,732.878333,86970.623333,2025-03-27,87020.751502,-1
2025-03-27 17:00:00,86835.24,87150.95,86796.00,86913.05,120897,50367,70530,596.49192,257.41179,339.08013,...,1,354.95,315.71,39.24,354.95,703.821667,86953.333333,2025-03-27,87018.010424,-1


In [23]:
validate_frame = df_filtered[
    (df_filtered.index >= pd.to_datetime('2025-01-01').tz_localize(None)) &
    (df_filtered.index < pd.to_datetime('2025-02-28').tz_localize(None))
]

In [24]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(validate_frame)
e.add_strategy(MultiStrategyAggregator(strategy_dict)) # Pass both df_filtered and strategy_dict
stats = e.run()

 36%|███▋      | 508/1392 [00:00<00:00, 2416.49it/s]

2025-01-10 18:00:00 New position opened: <Position: btc size: 0.15658756 entry: 95793.05>,transaction fee: 14.999999964457999
2025-01-10 22:00:00 Stop Loss Filled. Limit: 94659.21666666667 / Low: 94586.03
2025-01-10 22:00:00 Position closed for ticker btc at 94659.21666666667, transaction fee: 14.822455769344666
2025-01-11 15:00:00 New position opened: <Position: btc size: 0.15852885 entry: 94620.0>,transaction fee: 14.999999787
2025-01-11 16:00:00 Stop Loss Filled. Limit: 94360.62833333333 / Low: 94312.56
2025-01-11 16:00:00 Position closed for ticker btc at 94360.62833333333, transaction fee: 14.958881894960749
2025-01-16 20:00:00 New position opened: <Position: btc size: 0.1493768 entry: 100417.2>,transaction fee: 15.000000000960002
2025-01-16 22:00:00 Stop Loss Filled. Limit: 99601.28333333333 / Low: 99377.0
2025-01-16 22:00:00 Position closed for ticker btc at 99601.28333333333, transaction fee: 14.878120980226667
2025-01-19 02:00:00 New position opened: <Position: btc size: 0.143

100%|██████████| 1392/1392 [00:00<00:00, 2531.67it/s]

2025-02-11 05:00:00 New position opened: <Position: btc size: 0.15295988 entry: 98064.93>,transaction fee: 14.9999999250084
2025-02-11 12:00:00 Stop Loss Filled. Limit: 97775.18333333332 / Low: 97680.49
2025-02-11 12:00:00 Position closed for ticker btc at 97775.18333333332, transaction fee: 14.955680309644665
2025-02-14 22:00:00 New position opened: <Position: btc size: 0.15391437 entry: 97456.79>,transaction fee: 15.0000004350723
2025-02-15 03:00:00 Take Profit Filled. Limit: 97943.11166666666 / High: 97972.26
2025-02-15 03:00:00 Position closed for ticker btc at 97943.11166666666, transaction fee: 15.07485232801465
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      -0.68
Buy-and-Hold Total Return (%)            |      -9.57
Average Exposure to Asset (%)            |       0.26
Strategy CAGR (%)                        |      -4.30
Buy & Hold CAGR (%)                      |     -

In [25]:
e.plot()

In [26]:
## Optimising with optuna
train_data = df_filtered.loc["2024-03-02":"2024-12-31"]
val_data = df_filtered.loc["2025-01-01":"2025-02-28"]

import optuna
import pandas as pd
import numpy as np
import optuna.visualization as vis
import os

# Set your directory
output_dir = "../data/optuna_backtest_results"
os.makedirs(output_dir, exist_ok=True)  


signal_names = [
    'EMA_Signal', 'MACD_Signal', 'BB_RSI_Signal',
    'GoldenCross_Signal', 'Stochastic_Signal', 'OBV_Signal', 'VWAP_Signal',
    # 'reddit_Signal',
    # 'news_Signal',
    # 'tele_Signal'
]

# For tracking all trials' results
all_trials = []

def objective(trial):
    weights = [trial.suggest_float(name, 0, 1) for name in signal_names]

    # Normalize
    total = sum(weights)
    if total == 0:
        return -np.inf
    weights = [w / total for w in weights]

    strategy_dict = dict(zip(signal_names, weights))

    # Suggest additional hyperparameters
    bet_size = trial.suggest_float("bet_size", 0.5, 5.0)  # Position size per trade
    atr_multiplier = trial.suggest_float("atr_multiplier", 0.5, 5.0)  # TP/SL sensitivity
    use_leverage = trial.suggest_categorical("use_leverage", [True, False])  # Binary toggle

    # Run backtest
    e = Engine()
    e.add_data(train_data)
    e.add_strategy(MultiStrategyAggregator(
        weights=strategy_dict, 
        atr_multiplier=atr_multiplier,
        ))
    stats = e.run()

    short_weights = "_".join([
        f"{k[:4]}{int(v*100)}" for k, v in strategy_dict.items()
    ])
    plot_name = os.path.join(output_dir, f"trial_{trial.number}_{short_weights}.html")

    # # Save interactive Plotly chart
    # fig = e.plot(show_signals=False)  # modify plot() to return the figure object
    # fig.write_html(plot_name)

    # Save trial data
    trial_result = {
        "trial": trial.number,
        "sharpe": stats.get("Strategy Sharpe Ratio", np.nan),
        "return": stats.get("Total Return (%)", np.nan),
        "drawdown": stats.get("Strategy Max Drawdown (%)", np.nan),
        "atr_multiplier": atr_multiplier,
        "use_leverage": use_leverage,
        "plot_path": plot_name,
        **strategy_dict
    }
    all_trials.append(trial_result)

    return trial_result["sharpe"]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Save all trial results
results_df = pd.DataFrame(all_trials)
results_df.to_csv("../data/optuna_backtest_results/optuna_results_train_new.csv", index=False)

print("Best Sharpe:", study.best_value)
print("Best weights:", study.best_params)

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()


c:\Users\leoqi\miniforge3\envs\Finance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

[I 2025-03-28 18:05:03,891] A new study created in memory with name: no-name-50c84e79-fefa-4505-a4b9-7524a664405a
 13%|█▎        | 956/7320 [00:00<00:02, 2262.41it/s]

2024-03-25 13:00:00 New position opened: <Position: btc size: 0.22361363 entry: 67079.99>,transaction fee: 15.000000064263702
2024-03-25 14:00:00 Take Profit Filled. Limit: 67742.70525517515 / High: 69450.0
2024-03-25 14:00:00 Position closed for ticker btc at 67742.70525517515, transaction fee: 15.148192228129794


 23%|██▎       | 1706/7320 [00:00<00:02, 2428.84it/s]

2024-04-21 02:00:00 New position opened: <Position: btc size: 0.23096111 entry: 64946.0>,transaction fee: 15.000000250060001
2024-04-21 03:00:00 Take Profit Filled. Limit: 65358.612473457346 / High: 65695.56
2024-04-21 03:00:00 Position closed for ticker btc at 65358.612473457346, transaction fee: 15.095297684929553


 31%|███       | 2265/7320 [00:00<00:01, 2617.26it/s]

...

2024-11-21 04:00:00 New position opened: <Position: btc size: 0.15657927 entry: 95798.12>,transaction fee: 14.999999696972399
2024-11-21 12:00:00 Take Profit Filled. Limit: 98150.92113788002 / High: 98250.0
2024-11-21 12:00:00 Position closed for ticker btc at 98150.92113788002, transaction fee: 15.368399581596822
2024-11-29 02:00:00 New position opened: <Position: btc size: 0.15638032 entry: 95920.0>,transaction fee: 15.0000002944
2024-11-30 19:00:00 New position opened: <Position: btc size: 0.17814986 entry: 96966.37>,transaction fee: 17.274545240208198
2024-12-01 21:00:00 Take Profit Filled. Limit: 97413.00628773154 / High: 97780.0
2024-12-01 21:00:00 Position closed for ticker btc at 97413.00628773154, transaction fee: 15.23347709543747
2024-12-03 09:00:00 Stop Loss Filled. Limit: 95244.66617672773 / Low: 95158.0
2024-12-03 09:00:00 Position closed for ticker btc at 95244.66617672773, transaction fee: 16.967823945130778
2024-12-04 20:00:00 New position opened: <Position: btc size: 

100%|██████████| 7320/7320 [00:03<00:00, 2074.65it/s]

2024-12-12 18:00:00 New position opened: <Position: btc size: 0.14761215 entry: 101617.65>,transaction fee: 14.999999794447499
2024-12-13 07:00:00 Stop Loss Filled. Limit: 99633.3173186171 / Low: 99500.0
2024-12-13 07:00:00 Position closed for ticker btc at 99633.3173186171, transaction fee: 14.707088181033305



[I 2025-03-28 18:09:02,793] Trial 49 finished with value: 0.060984140180594434 and parameters: {'EMA_Signal': 0.6457363598722078, 'MACD_Signal': 0.6797647588192339, 'BB_RSI_Signal': 0.23048351917479598, 'GoldenCross_Signal': 0.8902334419522218, 'Stochastic_Signal': 0.6727709380733966, 'OBV_Signal': 0.14417394510761383, 'VWAP_Signal': 0.4037484316560831, 'bet_size': 1.8720014894942494, 'atr_multiplier': 4.214198621800625, 'use_leverage': False}. Best is trial 7 with value: 10.531981269107034.


Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |       0.03
Buy-and-Hold Total Return (%)            |      49.84
Average Exposure to Asset (%)            |       2.88
Strategy CAGR (%)                        |       0.03
Buy & Hold CAGR (%)                      |      62.98
Strategy Volatility (%)                  |       0.57
Buy & Hold Volatility (%)                |       8.99
Strategy Sharpe Ratio                    |       0.06
Buy & Hold Sharpe Ratio                  |       7.00
Strategy Max Drawdown (%)                |      -4.21
Buy & Hold Max Drawdown (%)              |     -32.31
Number of Trades                         |      80.00
Number of Buys                           |      40.00
Number of Sells                          |      40.00
Number of Take Profits                   |      20.00
Number of Stop Losses                    |      20.00
Best Sharpe: 10.5319812691

In [27]:
best_params = study.best_params
strategy_dict = {k: v for k, v in best_params.items() if k in signal_names}

e = Engine()
e.add_data(val_data)  # ✅ Validation data
e.add_strategy(MultiStrategyAggregator(
    weights=strategy_dict,
    bet_size=best_params["bet_size"],
    atr_multiplier=best_params["atr_multiplier"],
    use_leverage=best_params["use_leverage"]
))
val_stats = e.run()
fig = e.plot()
#fig.write_html("validation_results.html")

  9%|▊         | 121/1416 [00:00<00:02, 609.30it/s]

2025-01-01 03:00:00 New position opened: <Position: btc size: 0.15940673 entry: 94098.91>,transaction fee: 14.999999539664302
2025-01-01 15:00:00 New position opened: <Position: btc size: 0.18318744 entry: 94175.89>,transaction fee: 17.2518401988216
2025-01-01 16:00:00 New position opened: <Position: btc size: 0.210232 entry: 94432.94>,transaction fee: 19.85282584208
2025-01-01 18:00:00 New position opened: <Position: btc size: 0.24225457 entry: 94143.85>,transaction fee: 22.806777899894502
2025-01-01 19:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 20:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 21:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 00:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 01:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 03:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 05:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 08:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-02 09:00

 16%|█▌        | 229/1416 [00:00<00:01, 820.47it/s]

2025-01-06 02:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-06 03:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-06 08:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-06 12:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-06 14:00:00 Take Profit Filled. Limit: 100825.91866962316 / High: 101295.23
2025-01-06 14:00:00 Position closed for ticker btc at 100825.91866962316, transaction fee: 29.845433805472567
2025-01-06 14:00:00 Take Profit Filled. Limit: 100657.08866962316 / High: 101295.23
2025-01-06 14:00:00 Position closed for ticker btc at 100657.08866962316, transaction fee: 27.170350832886303
2025-01-06 14:00:00 Take Profit Filled. Limit: 100887.00866962317 / High: 101295.23
2025-01-06 14:00:00 Position closed for ticker btc at 100887.00866962317, transaction fee: 23.95015818595265
2025-01-06 14:00:00 New position opened: <Position: btc size: 0.2712654 entry: 99492.51>,transaction fee: 26.988875522153997
2025-01-06 19:00:00 New position opened: <Position

 27%|██▋       | 381/1416 [00:00<00:01, 598.54it/s]

...

2025-02-12 09:00:00 Stop Loss Filled. Limit: 96020.1289699265 / Low: 96014.69
2025-02-12 09:00:00 Position closed for ticker btc at 96020.1289699265, transaction fee: 24.92973393029942
2025-02-12 13:00:00 Stop Loss Filled. Limit: 95152.2904100486 / Low: 94088.23
2025-02-12 13:00:00 Position closed for ticker btc at 95152.2904100486, transaction fee: 19.491723082615994
2025-02-12 13:00:00 Stop Loss Filled. Limit: 95232.0904100486 / Low: 94088.23
2025-02-12 13:00:00 Position closed for ticker btc at 95232.0904100486, transaction fee: 22.42215855914082
2025-02-12 13:00:00 Stop Loss Filled. Limit: 95647.6504100486 / Low: 94088.23
2025-02-12 13:00:00 Position closed for ticker btc at 95647.6504100486, transaction fee: 25.834925830583252
2025-02-12 18:00:00 New position opened: <Position: btc size: 0.15407547 entry: 97354.89>,transaction fee: 15.0000004335483
2025-02-12 20:00:00 New position opened: <Position: btc size: 0.17742427 entry: 97205.06>,transaction fee: 17.246536810806198
2025-02-

 86%|████████▌ | 1221/1416 [00:02<00:00, 564.81it/s]

2025-02-17 13:00:00 New position opened: <Position: btc size: 0.20523344 entry: 96457.76>,transaction fee: 19.796357899494396
2025-02-17 15:00:00 Stop Loss Filled. Limit: 95606.31941077086 / Low: 95564.0
2025-02-17 15:00:00 Position closed for ticker btc at 95606.31941077086, transaction fee: 14.730588598184642
2025-02-17 15:00:00 New position opened: <Position: btc size: 0.23546867 entry: 96799.99>,transaction fee: 22.7933649013133
2025-02-18 04:00:00 New position opened: <Position: btc size: 0.24864411 entry: 96200.0>,transaction fee: 23.919563382
2025-02-18 08:00:00 Stop Loss Filled. Limit: 95392.63066421052 / Low: 95050.0
2025-02-18 08:00:00 Position closed for ticker btc at 95392.63066421052, transaction fee: 16.924967858977165
2025-02-19 02:00:00 New position opened: <Position: btc size: 0.26050291 entry: 95479.99>,transaction fee: 24.872815241770898
2025-02-19 03:00:00 Buy Entry Not Filled. Insufficient cash.
2025-02-19 07:00:00 Buy Entry Not Filled. Insufficient cash.
2025-02-1

100%|██████████| 1416/1416 [00:02<00:00, 608.78it/s]

2025-02-23 22:00:00 Stop Loss Filled. Limit: 95540.10683977966 / Low: 95522.78
2025-02-23 22:00:00 Position closed for ticker btc at 95540.10683977966, transaction fee: 17.054872115177616
2025-02-23 22:00:00 Stop Loss Filled. Limit: 95540.71683977966 / Low: 95522.78
2025-02-23 22:00:00 Position closed for ticker btc at 95540.71683977966, transaction fee: 19.61313433629144
2025-02-24 22:00:00 Stop Loss Filled. Limit: 92837.11199297938 / Low: 92368.0
2025-02-24 22:00:00 Position closed for ticker btc at 92837.11199297938, transaction fee: 14.41689421697407
2025-02-25 01:00:00 New position opened: <Position: btc size: 0.16329678 entry: 91857.29>,transaction fee: 14.9999996765262
2025-02-25 06:00:00 New position opened: <Position: btc size: 0.18741343 entry: 92070.34>,transaction fee: 17.2552182206662
2025-02-25 07:00:00 Stop Loss Filled. Limit: 88228.53107743051 / Low: 88200.0
2025-02-25 07:00:00 Position closed for ticker btc at 88228.53107743051, transaction fee: 14.407435029074334
2025

In [28]:
best_params

{'EMA_Signal': 0.3012366419441862,
 'MACD_Signal': 0.20225802969719153,
 'BB_RSI_Signal': 0.31990594499436853,
 'GoldenCross_Signal': 0.6836267659113341,
 'Stochastic_Signal': 0.5654739246326571,
 'OBV_Signal': 0.4025513424949032,
 'VWAP_Signal': 0.2522880673548794,
 'bet_size': 2.1823800302239094,
 'atr_multiplier': 4.718045553035888,
 'use_leverage': False}